<a href="https://colab.research.google.com/github/Hassouna9/Water-Segmentaion-of-Sattalllite-Images-using-Deeplap3/blob/main/Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gdown
import gdown
!gdown --id 1vg8S73P3_uqvpT_QXFQElbJ7wh6vdIPk
!pip install rasterio

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1vg8S73P3_uqvpT_QXFQElbJ7wh6vdIPk
From (redirected): https://drive.google.com/uc?id=1vg8S73P3_uqvpT_QXFQElbJ7wh6vdIPk&confirm=t&uuid=19b400f8-95c7-4cba-a532-41869e97fa82
To: /content/data-20240912T162055Z-001.zip
100% 68.6M/68.6M [00:01<00:00, 35.4MB/s]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 28.2 MB/s eta 0:00:00


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, utils
import os
from PIL import Image
import matplotlib.pyplot as plt
import zipfile
with zipfile.ZipFile('/content/data-20240912T162055Z-001.zip', 'r') as zip_ref:
    zip_ref.extractall()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
from torch.utils.data import Dataset
import os

class SatelliteDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = [f for f in os.listdir(image_dir) if f.endswith('.tif')]

    def open_tiff(self, path):
        import rasterio  # Ensure rasterio is imported here
        with rasterio.open(path) as src:
            img = src.read()
            return np.moveaxis(img, 0, -1)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx].replace('.tif', '.png'))

        image = self.open_tiff(img_path)
        mask = Image.open(mask_path).convert('L')  # Assuming PIL Image is imported globally

        if self.transform:
            transformed = self.transform(image=image, mask=np.array(mask))
            image = transformed['image']
            mask = transformed['mask']

        return image, mask

    def __len__(self):
        return len(self.images)

In [ ]:
import torch
from torchvision import models
import torchvision.models.segmentation as segmentation

def create_deeplabv3(output_channels, in_channels):
    # Use a lighter backbone if possible, like ResNet50
    model = segmentation.deeplabv3_resnet50(weights=segmentation.DeepLabV3_ResNet50_Weights.DEFAULT)
    # Adjust the input and output layers accordingly
    model.backbone.conv1 = torch.nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
    model.classifier[4] = torch.nn.Conv2d(256, output_channels, kernel_size=1)
    model.aux_classifier[4] = torch.nn.Conv2d(256, output_channels, kernel_size=1)
    return model

In [ ]:
import torch
import matplotlib.pyplot as plt
import os
import numpy as np
import torchvision
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
from PIL import Image
import os

def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)

def load_checkpoint(checkpoint, model):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])

def get_loaders(train_dir, train_maskdir, val_dir, val_maskdir, batch_size, train_transform, val_transform, num_workers=2, pin_memory=False):
    train_ds = SatelliteDataset(
        image_dir=train_dir,
        mask_dir=train_maskdir,
        transform=train_transform
    )
    train_loader = DataLoader(train_ds, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory, shuffle=True)

    val_ds = SatelliteDataset(
        image_dir=val_dir,
        mask_dir=val_maskdir,
        transform=val_transform
    )
    val_loader = DataLoader(val_ds, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory, shuffle=False)

    return train_loader, val_loader

    return train_loader, val_loader
def calculate_iou(preds, targets, threshold=0.3):
    """ Calculate Intersection over Union (IoU) for water class (class = 1). """
    preds = (preds > threshold).float()  # Convert predictions to binary (water class vs background)
    intersection = (preds * targets).sum()
    union = (preds + targets).sum() - intersection
    iou = intersection / (union + 1e-8)  # Add epsilon to avoid division by zero
    return iou.item()

def calculate_precision(preds, targets, threshold=0.5):
    """ Calculate Precision for water class (class = 1). """
    preds = (preds > threshold).float()
    true_positives = (preds * targets).sum()
    predicted_positives = preds.sum()
    precision = true_positives / (predicted_positives + 1e-8)
    return precision.item()

def calculate_recall(preds, targets, threshold=0.5):
    """ Calculate Recall for water class (class = 1). """
    preds = (preds > threshold).float()
    true_positives = (preds * targets).sum()
    actual_positives = targets.sum()
    recall = true_positives / (actual_positives + 1e-8)
    return recall.item()

def calculate_f1(precision, recall):
    """ Calculate F1-score from precision and recall. """
    if precision + recall == 0:
        return 0.0
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1
def visualize_rgb(image_tensor, index=0, folder="rgb_visualization", file_prefix="image", red_band=3, green_band=2, blue_band=1):
    """ Visualize the multispectral image as an RGB image by combining specified bands from a batched tensor. """
    os.makedirs(folder, exist_ok=True)  # Ensure directory exists

    # Check if the tensor is on CPU or move it to CPU
    if image_tensor.is_cuda:
        image_tensor = image_tensor.cpu()

    # Extract the RGB bands from the specified image in the batch
    red = image_tensor[index, red_band, :, :].numpy()
    green = image_tensor[index, green_band, :, :].numpy()
    blue = image_tensor[index, blue_band, :, :].numpy()

    # Normalize the bands to the [0, 1] range for visualization (if necessary)
    red = (red - red.min()) / (red.max() - red.min())
    green = (green - green.min()) / (green.max() - green.min())
    blue = (blue - blue.min()) / (blue.max() - blue.min())

    # Stack the channels to create an RGB image
    rgb_image = np.stack([red, green, blue], axis=-1)

    # Plot and save the RGB image
    plt.figure()
    plt.imshow(rgb_image)
    plt.title("RGB Visualization")
    plt.axis('off')  # Turn off axis labels for clean visualization

    file_path = os.path.join(folder, f"{file_prefix}_{index}_rgb.png")
    plt.savefig(file_path)
    plt.close()

    print(f"RGB image visualized and saved at {file_path}")


def check_accuracy(loader, model, is_validation=True, device="cuda"):
    model.eval()  # Always set the model to evaluation mode for accuracy check

    num_correct = 0
    num_pixels = 0
    dice_score = 0
    total_iou = 0
    total_precision = 0
    total_recall = 0
    total_f1 = 0
    count = 0  # To track the number of batches for averaging

    with torch.no_grad():  # Disable gradient calculation
        for x, y in loader:
            x = x.to(device)
            y = y.to(device).unsqueeze(1)  # Ensure mask has the same shape as the output

            # Get model predictions
            outputs = model(x)
            preds = outputs['out']  # Accessing the primary output
            preds = torch.sigmoid(preds)  # Apply sigmoid to obtain probabilities
            preds = (preds > 0.5).float()  # Convert probabilities to binary output


            # Calculate the number of correct pixels
            num_correct += (preds == y).sum().item()
            num_pixels += torch.numel(preds)

            # Calculate the Dice score
            dice_score += (2 * (preds * y).sum()) / ((preds + y).sum() + 1e-8)

            # Compute metrics for each batch
            iou = calculate_iou(preds, y)
            precision = calculate_precision(preds, y)
            recall = calculate_recall(preds, y)
            f1 = calculate_f1(precision, recall)

            # Update totals for averaging later
            total_iou += iou
            total_precision += precision
            total_recall += recall
            total_f1 += f1
            count += 1

    # Calculate average metrics across all batches
    avg_iou = total_iou / count
    avg_precision = total_precision / count
    avg_recall = total_recall / count
    avg_f1 = total_f1 / count

    # Calculate accuracy as a percentage
    accuracy = num_correct / num_pixels * 100

    print(f"Got {num_correct}/{num_pixels} correct with accuracy {accuracy:.2f}%")
    print(f"Average IoU: {avg_iou:.4f}")
    print(f"Average Precision: {avg_precision:.4f}")
    print(f"Average Recall: {avg_recall:.4f}")
    print(f"Average F1 Score: {avg_f1:.4f}")

    model.train()  # Return the model to training mode if it's not the validation set


def save_tensor_as_image(tensor, folder, filename):
    """Saves a tensor as an image."""
    tensor = tensor.to('cpu')  # Move tensor to CPU
    grid = make_grid(tensor)  # Create a grid from tensor if batched

    # Scale to [0, 255], round off to the nearest integer, and clamp
    ndarr = grid.mul(255).add(0.5).clamp(0, 255)
    ndarr = ndarr.to(torch.uint8)  # Convert to uint8 only after all modifications

    # Permute dimensions to match image format
    ndarr = ndarr.permute(1, 2, 0)

    # Handle potential issues with shape, e.g., single channel grayscale images
    if ndarr.shape[2] == 1:
        ndarr = ndarr.squeeze(2)  # If only one channel, remove the channel dimension

    im = Image.fromarray(ndarr.numpy(), 'L' if ndarr.ndim == 2 else 'RGB')  # Specify mode based on dimensions
    os.makedirs(folder, exist_ok=True)  # Ensure the directory exists
    im.save(os.path.join(folder, f'{filename}.png'))  # Save the image
    print(f"Saved image to {os.path.join(folder, f'{filename}.png')}")  # Confirmation message

def _save_single_image(tensor, folder, filename):
    """ Helper function to save a single image tensor. """
    plt.figure()
    if tensor.shape[0] == 1:  # Grayscale
        plt.imshow(tensor.squeeze(0).cpu().numpy(), cmap='gray')
    elif tensor.shape[0] <= 4:  # Standard RGB or RGBA
        plt.imshow(tensor.permute(1, 2, 0).cpu().numpy())
    else:  # More than 4 channels, visualize the first three as RGB
        plt.imshow(tensor[:3].permute(1, 2, 0).cpu().numpy())
    plt.axis('off')
    plt.savefig(os.path.join(folder, filename))
    plt.close()

def save_predictions_as_imgs(loader, model, folder="saved_images/", device="cuda"):
    model.eval()  # Set model to evaluation mode
    for idx, (x, y) in enumerate(loader):
        x = x.to(device)

        with torch.no_grad():
            outputs = model(x)  # Model outputs are expected to be a dictionary
            preds = outputs['out']  # Extract the tensor associated with the key 'out'

            preds = torch.sigmoid(preds)  # Apply sigmoid to convert to probabilities
            preds = (preds > 0.5).float()  # Threshold to get binary mask
            preds = (preds * 255).byte()  # Scale to 0-255 and convert to uint8 for visualization or storage

        # Ensure the folder exists before trying to save files
        os.makedirs(folder, exist_ok=True)
        save_tensor_as_image(preds, folder, f"pred_{idx}")  # Save the predictions

        # Adjust and save ground truth images if necessary
        if y.dim() == 4:
            y = y.squeeze(1)  # Reduce it to [batch, height, width] if necessary
        save_tensor_as_image(y * 255, folder, f"y_{idx}")  # Scale ground truth for consistency with predictions

    model.train()  # Set model back to training mode if needed




In [ ]:
import torch
import os
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import torch.amp


def define_transforms():
    # Adjust transforms to use smaller dimensions and efficient processing
    train_transform = A.Compose([
        # A.Resize(256, 256),  # Reduced from 513x513 to 256x256
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])
    val_transforms = A.Compose([
        # A.Resize(256, 256),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])
    return train_transform, val_transforms

def train_fn(model, dataloader, optimizer, loss_fn, device, epoch, num_epochs):
    model.train()
    loop = tqdm(dataloader, leave=True)
    for images, targets in loop:
        images = images.to(device)
        targets = targets.to(device).unsqueeze(1).float()  # Ensure targets are float

        optimizer.zero_grad()
        output = model(images)['out']

        # Ensure output is float; typically, it should already be
        if output.dtype != torch.float32:
            output = output.float()

        loss = loss_fn(output, targets)
        loss.backward()
        optimizer.step()

        loop.set_description(f"Epoch [{epoch+1}/{num_epochs}]")
        loop.set_postfix(loss=loss.item())

def main():
    LOAD_MODEL = False
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Running on device: {DEVICE}")

    model = create_deeplabv3(output_channels=1, in_channels=12).to(DEVICE)  # Assuming you are using multi-spectral data with 12 channels
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    loss_fn = nn.BCEWithLogitsLoss()  # Ensure that targets are appropriately shaped

    scaler = torch.amp.GradScaler() if DEVICE == 'cuda' else None

    train_transform, val_transforms = define_transforms()
    train_loader, val_loader = get_loaders(
    "data/train_images/", "data/train_masks/",
    "data/val_images/", "data/val_masks/",
    8,  # Adjusted batch size, now as a positional argument
    train_transform, val_transforms,
    num_workers=2,  # Reduced number of worker threads, now as a keyword argument
    pin_memory=False  # Disabled pin_memory to conserve RAM, now as a keyword argument
    )

    first_batch_images, first_batch_labels = next(iter(train_loader))
    visualize_rgb(first_batch_images, index=0)  # You can change index to visualize different images in the batch

    if LOAD_MODEL and os.path.isfile("my_checkpoint.pth.tar"):
        load_checkpoint(torch.load("my_checkpoint.pth.tar"), model)

    num_epochs = 1  # Define this before the loop
    for epoch in range(num_epochs):
        train_fn(model, train_loader, optimizer, loss_fn, DEVICE, epoch, num_epochs)
        check_accuracy(val_loader, model, device=DEVICE)
        if epoch % 5 == 0:  # Optionally save the model every 5 epochs
            save_checkpoint({'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict()})

    save_predictions_as_imgs(val_loader, model, folder="saved_images", device=DEVICE)

if __name__ == "__main__":
    main()


Running on device: cuda


/usr/local/lib/python3.10/dist-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/usr/local/lib/python3.10/dist-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


RGB image visualized and saved at rgb_visualization/image_0_rgb.png


  0%|          | 0/34 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/usr/local/lib/python3.10/dist-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
Epoch [1/1]: 100%|██████████| 34/34 [00:06<00:00,  4.88it/s, loss=0.462]
/usr/local/lib/python3.10/dist-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/usr/local/lib/python3.10/dist-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity 

Got 432269/589824 correct with accuracy 73.29%
Average IoU: 0.5274
Average Precision: 0.6172
Average Recall: 0.8084
Average F1 Score: 0.6679
=> Saving checkpoint


/usr/local/lib/python3.10/dist-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/usr/local/lib/python3.10/dist-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


Saved image to saved_images/pred_0.png
Saved image to saved_images/y_0.png
Saved image to saved_images/pred_1.png
Saved image to saved_images/y_1.png
Saved image to saved_images/pred_2.png
Saved image to saved_images/y_2.png
Saved image to saved_images/pred_3.png
Saved image to saved_images/y_3.png
Saved image to saved_images/pred_4.png
Saved image to saved_images/y_4.png
